In [27]:
# Import necessary libraries

import requests
import lxml.html as lh
import pandas as pd

In [28]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [29]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [30]:
# Parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [37]:
len(tr_elements)

294

In [39]:
# Creating Pandas DataFrame

for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [41]:
col

[('Postcode',
  ['M1A',
   'M2A',
   'M3A',
   'M4A',
   'M5A',
   'M5A',
   'M6A',
   'M6A',
   'M7A',
   'M8A',
   'M9A',
   'M1B',
   'M1B',
   'M2B',
   'M3B',
   'M4B',
   'M4B',
   'M5B',
   'M5B',
   'M6B',
   'M7B',
   'M8B',
   'M9B',
   'M9B',
   'M9B',
   'M9B',
   'M9B',
   'M1C',
   'M1C',
   'M1C',
   'M2C',
   'M3C',
   'M3C',
   'M4C',
   'M5C',
   'M6C',
   'M7C',
   'M8C',
   'M9C',
   'M9C',
   'M9C',
   'M9C',
   'M1E',
   'M1E',
   'M1E',
   'M2E',
   'M3E',
   'M4E',
   'M5E',
   'M6E',
   'M7E',
   'M8E',
   'M9E',
   'M1G',
   'M2G',
   'M3G',
   'M4G',
   'M5G',
   'M6G',
   'M7G',
   'M8G',
   'M9G',
   'M1H',
   'M2H',
   'M3H',
   'M3H',
   'M3H',
   'M4H',
   'M5H',
   'M5H',
   'M5H',
   'M6H',
   'M6H',
   'M7H',
   'M8H',
   'M9H',
   'M1J',
   'M2J',
   'M2J',
   'M2J',
   'M3J',
   'M3J',
   'M4J',
   'M5J',
   'M5J',
   'M5J',
   'M6J',
   'M6J',
   'M7J',
   'M8J',
   'M9J',
   'M1K',
   'M1K',
   'M1K',
   'M2K',
   'M3K',
   'M3K',
   'M4K',
   'M4

In [6]:
# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

[288, 288, 288]

In [42]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [43]:
# Access the top 5 rows of the data frame 
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [48]:
df.columns = [ 'Neighbourhood','Postcode', 'Borough']
cols = df.columns.tolist()
cols
# cols = cols[-1:] + cols[:-1]
df = df[cols]
df.head()

,Neighbourhood,Postcode,Borough
0,Not assigned\n,M1A,Not assigned
1,Not assigned\n,M2A,Not assigned
2,Parkwoods\n,M3A,North York
3,Victoria Village\n,M4A,North York
4,Harbourfront\n,M5A,Downtown Toronto


In [49]:
df = df.replace('\n',' ', regex=True)
df.head()

,Neighbourhood,Postcode,Borough
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,Harbourfront,M5A,Downtown Toronto


In [50]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,Neighbourhood,Postcode,Borough
0,Parkwoods,M3A,North York
1,Victoria Village,M4A,North York
2,Harbourfront,M5A,Downtown Toronto
3,Regent Park,M5A,Downtown Toronto
4,Lawrence Heights,M6A,North York
5,Lawrence Manor,M6A,North York
6,Not assigned,M7A,Queen's Park
7,Islington Avenue,M9A,Etobicoke
8,Rouge,M1B,Scarborough
9,Malvern,M1B,Scarborough


In [51]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park ,Ionview ,Kennedy Park"
7,M1L,Scarborough,"Clairlea ,Golden Mile ,Oakridge"
8,M1M,Scarborough,"Cliffcrest ,Cliffside ,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff ,Cliffside West"


In [52]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

In [53]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [54]:
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [55]:
df.shape

(103, 3)

In [56]:
df.to_csv(r'df_can.csv')